## QCoDeS Training for Developers

Welcome to the training about QCoDeS targeted for developers. After this training, developers should have more knowledge about QCoDeS which will bring them confidence with contributing to it.

### Approach

- Don't bring too much relevant background, only small absolutely necessary portions
- Provide refresh of user-side view on QCoDeS
- Describe features/concepts from ideas, but get close to code very quickly

### Execution plan for presenter

- Before:
  - Have qcodes code at hand in your IDE https://github.com/qcodes/qcodes
  - Run the training Jupyter notebook, and have it at hand https://github.com/astafan8/Qcodes/blob/training-for-devs/docs/examples/QCoDeS%20Training%20for%20Developers.ipynb
  - Familiarize yourself with the following info from the docs:
    - Front page https://qcodes.github.io/Qcodes/index.html
    - 15 minutes of QCoDeS https://qcodes.github.io/Qcodes/examples/15_minutes_to_QCoDeS.html
    - `Parameter`s
      - overview1 https://qcodes.github.io/Qcodes/examples/Parameters/Parameters.html
      - overview2 https://qcodes.github.io/Qcodes/examples/writing_drivers/Creating-Instrument-Drivers.html#What%E2%80%99s-a-Parameter? - only this section
      - API docs https://qcodes.github.io/Qcodes/api/parameters/parameter.html#module-qcodes.instrument.parameter
      - `ParameterWithSetpoints` https://qcodes.github.io/Qcodes/examples/Parameters/Simple-Example-of-ParameterWithSetpoints.html
    - Performing measurements https://qcodes.github.io/Qcodes/examples/DataSet/Performing-measurements-using-qcodes-parameters-and-dataset.html
    - dataset notebooks
      - the class https://qcodes.github.io/Qcodes/examples/DataSet/DataSet-class-walkthrough.html
      - data access https://qcodes.github.io/Qcodes/examples/DataSet/Accessing-data-in-DataSet.html
- During:
  - Go through notebook in 3 parts, have 5 min breaks, 5 min reserve beginning, 10 min reserve at the end
  - Have a time keeper
  - Let ask questions at specific moments but very frequently to feel interactive
- After:
  - Port notebook to qcodes docs ASAP
  - Sessions on some parts if requested

## QCoDeS? ah? [10min]

What is it? What are it's goals, and users? And why?

### "The It"

**Q.**uantum **Co**penhagen **De**lft **S**ydney:

- data acquisition framework
- modular
- python-based
- open-source

### Goal

**Common** framework for physics experiments:

- users need to write only their-own-experiment-specific code
- physics experiments can take advantage of modern software and best practices
- code can and should be contributed back to the framework
- the process of moving between teams or labs, and of setting up a new experiment is streamlined
- new students don’t need to spend a long time learning software in order to participate in experiments

### Users

- people doing measurements:
  - academics: e.g. scientists and students
  - corporates: e.g. experimentalists, engineers, hardware manufacturers
- worldwide

### Driven by

- Microsoft, largely
- worldwide comminity, growing

### Development principles

- Robust code - as opposed to what users are used to from other measurement frameworks
- Respect backwards compatibility - don't break user's code that "works"!
- Modularity - extend-existing, not go-off-and-write-your-own
- User facing API - list in docs, devs can use everything as appropriate
- Carefully scoping problems

## I wanna MEASURE [30min]

How do I measure things? How are measurements enabled via QCoDeS? Station, isntruments, parameters?

- Quickly about general stuff
- Deeper into parameters


	• Station - just a bucket of insrtuments, also yaml config
	• Instruments - just a bucket of parameters and hardware interaction
		○ Base vs not Base
		○ write/ask methods
		○ Visa vs DLL vs ???
		○ Registry of instrument instances - how does it work and how to see which instances exists etc.
		○ Submodules, Channels, channel lists
		○ Logging - logger instance on each instance
		○ Writing drivers notebook - skip
	• Parameters
		○ What they are - two purposes
		○ Name/label/unit/docstring - metadata
		○ Set/get
			§ _raw
			§ command
		○ Set_to
		○ Cache and get_latest
		○ Value/raw_value
			§ Parsers
			§ Val_mapping
			§ Validators
		○ Snapshot - it's nontriviallity and flags
		○ More specific parameters:
			§ Delegate parameter
			§ Array + setpoints
			§ Multiparameter
			§ Group parameter
			§ How to subclass in general
	• Snapshot - metadatable, snapshot/snapshot_base, etc

## BREAK [5min]

## I wanna STORE [30min]

How and where do I store what I measured? And access back? Database, experiments, datasets, parameters?

- Deeper in data saving approach
- Quickly DB structure, paramtypes
- mention DataSet is too close to sqlite


	• How people measure - interactively!
		○ Why loop and not loop anymore
		○ Measurement as general thing and create-your-own-measurement-function
		○ Dond, pysweep, etc.
	• Measurement -> DataSet
		○ Name, experiment, station
		○ Parameters
			§ Register_* - as interface between qcodes parameters and dataset parameters
			§ Rundescription/interdeps/paramspecbase
			§ paramtype
		○ Before and after actions :)
		○ .run() -> Runner -> DataSaver - what is their role?
			§ Runner
			§ DataSaver
				□ Add_results
					® Validation that happens
				□ Dataset/run_id
		○ Sqlite data saving optimizations
			§ Paramtype "array"
			§ wrtie_period
			§ write_in_background
		○ Subscribers - bonus topic
	• Database stuff
		○ Runs table as log of performed measurements
		○ Experiment as a bucket of datasets but not used as such really
			§ Ideas for "active experiment, etc"
		○ Data storage
			§ As a message queue/log - needs reconstruction
			§ in the same DB
		○ Sqlite database, schema, versions
			§ Connect - users might forget to close, etc
		○ Sqlite module overview
	• DataSet walkthrough
		○ Identity, also timestamps and state
		○ parameters
		○ Getting data
		○ Metadata feature


## BREAK [5min]

## I want MORE [5min]

What else does QCoDeS provide out of the box?

- q/a, other qcodes topics for self-exploration

- contrib-drivers
- Config
- Plotting support
- Deprecation
- Linking datasets
- Extracting runs/datasets from db to another
- Monitor
- Interactive widget
- Legacy, loop-entangled, features

## Presenter-only section

Some info, notes, and scratch-space for presenter.